# Purpose of this program 
Filling in missing Ozone data.

Not all monitors had PM and Ozone measurements.

Ozone is more reginal measurement.

So those missing Ozone in the Cleaned Air Quality matching, Ozone from other near by monitors can be used.

Locating the closest non-missing Ozone and matching it based on distance between the Sources and Outcomes.


In [1]:
import pandas as pd
# Import the Numpy dependency
import numpy as np
# Import the Haversine formula package.  Calculates distances using lat & log
import haversine as hs
from haversine import Unit

Bring in Outcome data (500 City) only keeping locational information for now

In [2]:
# Bringing in Cleaned Air Quality measurements
city_aq_file_path = "../ProcessedData/FinalAirQuality.csv"
cityaqdf = pd.read_csv(city_aq_file_path)

missozone = cityaqdf[cityaqdf['Ozone_Max'].isna()]
missozone.head()


,Place_TractID,StateAbbr,PlaceName,city_state,PlaceFIPS,TractFIPS,CountyFIPS,SiteId,PM2.5_Exceptional,PM2.5_ExceedCount,PM2.5_Max,Ozone_Exceptional,Ozone_ExceedCount,Ozone_Max
0,0107000-01073003200,AL,Birmingham,Birmingham_AL,107000,1073003200,1073,15.0,0,0.0,22.0,NaN,NaN,NaN
1,0107000-01073003300,AL,Birmingham,Birmingham_AL,107000,1073003300,1073,15.0,0,0.0,22.0,NaN,NaN,NaN
2,0107000-01073010500,AL,Birmingham,Birmingham_AL,107000,1073010500,1073,15.0,0,0.0,22.0,NaN,NaN,NaN
3,0107000-01073010701,AL,Birmingham,Birmingham_AL,107000,1073010701,1073,17.0,0,0.0,23.1,NaN,NaN,NaN
5,0107000-01073010802,AL,Birmingham,Birmingham_AL,107000,1073010802,1073,17.0,0,0.0,23.1,NaN,NaN,NaN


In [3]:
missing = missozone[['Place_TractID','StateAbbr','PlaceName','city_state','PlaceFIPS','TractFIPS','CountyFIPS']]

In [4]:
# Bringing in 500 city data 
city_file_path = "../ProcessedData/500_City_cleaned.csv"
citydf = pd.read_csv(city_file_path)
citydf.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,Insurance,HighBloodPressure,Cancer,Asthma,HeartDisease,AnnualCheckUps,Smokes,MentalHealthIssues,Latitude,Longitude,newFIPS,CountyFIPS
0,AL,Birmingham,107000,1073003200,0107000-01073003200,931.0,26.8,57.0,6.0,14.4,11.1,78.2,29.6,22.0,33.509402,-86.885908,1073,1073
1,AL,Birmingham,107000,1073003300,0107000-01073003300,947.0,21.4,55.2,6.8,12.9,9.7,80.1,22.6,16.7,33.517126,-86.891382,1073,1073
2,AL,Birmingham,107000,1073010500,0107000-01073010500,114.0,23.9,60.5,7.1,13.6,11.1,80.7,26.5,18.6,33.436379,-86.912892,1073,1073
3,AL,Birmingham,107000,1073010701,0107000-01073010701,74.0,19.8,24.9,2.7,11.2,2.4,66.2,18.1,16.1,33.473886,-86.814649,1073,1073
4,AL,Birmingham,107000,1073010801,0107000-01073010801,168.0,7.3,33.0,7.4,8.9,4.5,73.7,8.5,8.8,33.514098,-86.746697,1073,1073


In [5]:
cityloc = citydf[['StateAbbr','Place_TractID','Longitude', 'Latitude']]

Reading in Source and limiting it to select cities

In [6]:
# Reading in Cleaned AirQuality data
input_file_path = "../ProcessedData/AirQuality_cleaned.csv"
aqdf = pd.read_csv(input_file_path)
# Keep Non Missing Ozone monitors
aqdf = aqdf.dropna()
aqdf.head()

,SiteId,Latitude,Longitude,Sample Duration,PM2.5_Exceptional,PM2.5_ExceedCount,PM2.5_Max,State,County,City,index,Ozone_Exceptional,Ozone_ExceedCount,Ozone_Max,StateAbbr,city_state
0,11,33.553056,-86.815000,24-HR BLK AVG,0,0.0,26.6,Alabama,Jefferson,Birmingham,69,0.0,1.0,0.078,AL,Birmingham_AL
3,25,34.685470,-86.588160,24 HOUR,0,0.0,18.5,Alabama,Madison,Huntsville,500,0.0,0.0,0.070,AL,Huntsville_AL
4,29,32.412811,-86.263394,24 HOUR,0,0.0,20.8,Alabama,Montgomery,Montgomery,564,0.0,0.0,0.060,AL,Montgomery_AL
8,75,33.483780,-112.142560,24-HR BLK AVG,0,5.0,86.6,Arizona,Maricopa,Phoenix,1730,0.0,9.0,0.088,AZ,Phoenix_AZ
9,75,33.483780,-112.142560,24-HR BLK AVG,0,5.0,86.6,Arizona,Maricopa,Phoenix,1730,0.0,14.0,0.088,AZ,Phoenix_AZ


In [7]:
aqdf2 = aqdf[['StateAbbr','SiteId', 'Longitude', 'Latitude']]

In [8]:
aqdf2.describe()

,SiteId,Longitude,Latitude
count,156.000000,156.000000,156.000000
mean,1015.955128,-101.412552,36.672026
std,794.158588,15.929325,4.612607
min,11.000000,-157.871171,21.303382
25%,307.000000,-115.070445,33.487043
50%,948.000000,-98.524478,36.449581
75%,1662.500000,-87.509152,40.182130
max,2595.000000,-71.082600,47.568236


Joining data on County_FIPS

In [9]:
outerdf = aqdf2.merge(cityloc, how = 'outer', left_on="StateAbbr",  right_on = 'StateAbbr')
outerdf = outerdf.dropna(subset = ['StateAbbr'])
outerdf.describe()

,SiteId,Longitude_x,Latitude_x,Longitude_y,Latitude_y
count,13513.000000,13513.000000,13513.000000,13552.000000,13552.000000
mean,777.384519,-109.980896,35.042976,-109.030138,34.535155
std,769.554284,13.808957,3.778488,13.617436,3.448550
min,11.000000,-157.871171,21.303382,-158.112465,21.281244
25%,274.000000,-120.994212,32.845709,-118.214967,33.066287
50%,383.000000,-117.274063,34.510961,-117.211729,33.934657
75%,1323.000000,-98.291069,37.743065,-96.987150,36.746321
max,2595.000000,-71.082600,47.568236,-70.965134,61.217709


In [10]:
outerdf.head()

,StateAbbr,SiteId,Longitude_x,Latitude_x,Place_TractID,Longitude_y,Latitude_y
0,AL,11.0,-86.815,33.553056,0107000-01073003200,-86.885908,33.509402
1,AL,11.0,-86.815,33.553056,0107000-01073003300,-86.891382,33.517126
2,AL,11.0,-86.815,33.553056,0107000-01073010500,-86.912892,33.436379
3,AL,11.0,-86.815,33.553056,0107000-01073010701,-86.814649,33.473886
4,AL,11.0,-86.815,33.553056,0107000-01073010801,-86.746697,33.514098


In [11]:
outerdf['Sourcecoor'] = list(zip(outerdf.Latitude_x, outerdf.Longitude_x))
outerdf['Tractcoor'] = list(zip(outerdf.Latitude_y, outerdf.Longitude_y))

In [12]:
# Function from https://github.com/ashutoshb418/Foodies-Visualization/blob/master/Foodies_Chain.ipynb
def distance_from(loc1,loc2): 
    dist=hs.haversine(loc1,loc2)
    return round(dist,5)

In [13]:
# Calculate distances
outerdf['dist']=outerdf.apply(lambda row: distance_from(row.Sourcecoor,row.Tractcoor), axis = 1)

In [14]:
# Determine the min for each Air Quality site
outerdf['mindist'] = outerdf.groupby(outerdf.Place_TractID).dist.transform('min')
outerdf.head()

,StateAbbr,SiteId,Longitude_x,Latitude_x,Place_TractID,Longitude_y,Latitude_y,Sourcecoor,Tractcoor,dist,mindist
0,AL,11.0,-86.815,33.553056,0107000-01073003200,-86.885908,33.509402,"(33.553056, -86.815)","(33.5094018502, -86.8859081961)",8.17071,8.17071
1,AL,11.0,-86.815,33.553056,0107000-01073003300,-86.891382,33.517126,"(33.553056, -86.815)","(33.5171261108, -86.8913819749)",8.12909,8.12909
2,AL,11.0,-86.815,33.553056,0107000-01073010500,-86.912892,33.436379,"(33.553056, -86.815)","(33.4363786806, -86.9128923072)",15.83428,15.83428
3,AL,11.0,-86.815,33.553056,0107000-01073010701,-86.814649,33.473886,"(33.553056, -86.815)","(33.473886155, -86.8146487762)",8.80336,8.80336
4,AL,11.0,-86.815,33.553056,0107000-01073010801,-86.746697,33.514098,"(33.553056, -86.815)","(33.514097853, -86.7466971362)",7.67109,7.67109


In [16]:
# Limit to the closest site
df = outerdf[outerdf.dist == outerdf.mindist]


In [17]:
df = df.drop(['Sourcecoor','Longitude_x','Latitude_x'], axis=1)
# Find duplicates entries
print(f"Duplicate entries: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"Duplicate entries: {df.duplicated().sum()}")

Duplicate entries: 169
Duplicate entries: 0


In [18]:
df2 = df[['Place_TractID','SiteId']]
df2.describe()

,SiteId
count,1339.000000
mean,1181.879014
std,784.942927
min,11.000000
25%,400.000000
50%,1057.000000
75%,1920.000000
max,2595.000000


In [21]:
city = df2.merge(citydf, on = 'Place_TractID', how = 'left')
city.head()


,Place_TractID,SiteId,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Population2010,Insurance,HighBloodPressure,Cancer,Asthma,HeartDisease,AnnualCheckUps,Smokes,MentalHealthIssues,Latitude,Longitude,newFIPS,CountyFIPS
0,0107000-01073003200,11.0,AL,Birmingham,107000,1073003200,931.0,26.8,57.0,6.0,14.4,11.1,78.2,29.6,22.0,33.509402,-86.885908,1073,1073
1,0107000-01073003300,11.0,AL,Birmingham,107000,1073003300,947.0,21.4,55.2,6.8,12.9,9.7,80.1,22.6,16.7,33.517126,-86.891382,1073,1073
2,0107000-01073010500,11.0,AL,Birmingham,107000,1073010500,114.0,23.9,60.5,7.1,13.6,11.1,80.7,26.5,18.6,33.436379,-86.912892,1073,1073
3,0107000-01073010701,11.0,AL,Birmingham,107000,1073010701,74.0,19.8,24.9,2.7,11.2,2.4,66.2,18.1,16.1,33.473886,-86.814649,1073,1073
4,0107000-01073010801,11.0,AL,Birmingham,107000,1073010801,168.0,7.3,33.0,7.4,8.9,4.5,73.7,8.5,8.8,33.514098,-86.746697,1073,1073


In [22]:
# Merging on non-missing Ozone data

In [23]:
AQ = city[['StateAbbr','PlaceName','PlaceFIPS','TractFIPS','CountyFIPS','Place_TractID','SiteId']].merge(aqdf, on='SiteId', how = 'left')
AQ.columns

Index(['StateAbbr_x', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'CountyFIPS',
       'Place_TractID', 'SiteId', 'Latitude', 'Longitude', 'Sample Duration',
       'PM2.5_Exceptional', 'PM2.5_ExceedCount', 'PM2.5_Max', 'State',
       'County', 'City', 'index', 'Ozone_Exceptional', 'Ozone_ExceedCount',
       'Ozone_Max', 'StateAbbr_y', 'city_state'],
      dtype='object')

In [24]:
print(f"Duplicate entries: {AQ.duplicated().sum()}")
finalAQ = AQ.drop_duplicates()

Duplicate entries: 116


In [25]:
finalAQ.describe()

,PlaceFIPS,TractFIPS,CountyFIPS,SiteId,Latitude,Longitude,PM2.5_Exceptional,PM2.5_ExceedCount,PM2.5_Max,index,Ozone_Exceptional,Ozone_ExceedCount,Ozone_Max
count,1.392000e+03,1.392000e+03,1392.000000,1392.000000,1392.000000,1392.000000,1392.0,1392.000000,1392.000000,1392.000000,1392.0,1392.000000,1392.000000
mean,2.502230e+06,2.481872e+10,24818.573994,1149.479885,35.637211,-98.397350,0.0,1.380747,34.066164,32380.000000,0.0,7.557471,0.077190
std,1.770206e+06,1.763047e+10,17630.477383,788.961069,5.093851,15.711086,0.0,3.521752,21.981739,22237.408023,0.0,18.664725,0.013774
min,1.500300e+04,1.073003e+09,1073.000000,11.000000,21.303382,-157.871171,0.0,0.000000,11.800000,69.000000,0.0,0.000000,0.050000
25%,6.677500e+05,6.075980e+09,6075.000000,382.000000,32.805810,-112.191930,0.0,0.000000,22.200000,11636.000000,0.0,0.000000,0.070000
50%,2.148006e+06,2.111101e+10,21111.000000,1020.000000,35.240100,-95.326137,0.0,0.000000,27.500000,28027.000000,0.0,2.000000,0.076000
75%,4.082588e+06,4.036003e+10,40360.000000,1886.250000,39.751184,-84.504040,0.0,1.000000,41.100000,52718.250000,0.0,5.000000,0.080000
max,5.613900e+06,5.602100e+10,56021.000000,2595.000000,47.568236,-71.082600,0.0,29.000000,231.600000,71888.000000,0.0,112.000000,0.136000


In [27]:
# reorder & keep minimal number of variables

finaldf = finalAQ[['Place_TractID','SiteId', 'PM2.5_Exceptional', 
              'PM2.5_ExceedCount', 'PM2.5_Max','Ozone_Exceptional', 
              'Ozone_ExceedCount', 'Ozone_Max']]

finaldf.columns

Index(['Place_TractID', 'SiteId', 'PM2.5_Exceptional', 'PM2.5_ExceedCount',
       'PM2.5_Max', 'Ozone_Exceptional', 'Ozone_ExceedCount', 'Ozone_Max'],
      dtype='object')

In [28]:
finaldf = finaldf.groupby(finaldf.Place_TractID).last()
finaldf = finaldf.reset_index()
finaldf.head()

,Place_TractID,SiteId,PM2.5_Exceptional,PM2.5_ExceedCount,PM2.5_Max,Ozone_Exceptional,Ozone_ExceedCount,Ozone_Max
0,0107000-01073003200,11.0,0,0.0,26.6,0.0,1.0,0.078
1,0107000-01073003300,11.0,0,0.0,26.6,0.0,1.0,0.078
2,0107000-01073010500,11.0,0,0.0,26.6,0.0,1.0,0.078
3,0107000-01073010701,11.0,0,0.0,26.6,0.0,1.0,0.078
4,0107000-01073010801,11.0,0,0.0,26.6,0.0,1.0,0.078


In [29]:
# Saving Cleaned data into csv
output_file_path = "../ProcessedData/FinalAirQuality_nodups.csv"
finaldf.to_csv(output_file_path, index=False)

end of code

In [30]:
# check for multiple tracks
Place_TractIDdf = finaldf['Place_TractID']
print(f"Duplicate entries: {Place_TractIDdf.duplicated().sum()}")

Duplicate entries: 0
